In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # Принудительное перемонтирование

Mounted at /content/drive


In [ ]:
df_sessions = pd.read_csv('/content/drive/MyDrive/Проектный практикум (хакатон)/ga_sessions.csv', low_memory=False)

In [ ]:
df_sessions.shape

(1860042, 18)

In [ ]:
df_hits = pd.read_csv('/content/drive/MyDrive/Проектный практикум (хакатон)/ga_hits.csv', low_memory=False)

In [ ]:
df_hits.shape

(15726470, 11)

### **Обработка датасета df_sessions**

In [ ]:
# Просмотр колонок датасета
print(df_sessions.columns)

Index(['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'utm_keyword', 'device_category', 'device_os', 'device_brand',
       'device_model', 'device_screen_resolution', 'device_browser',
       'geo_country', 'geo_city'],
      dtype='object')


In [ ]:
# Подсчёт пропусков и сортировка по убыванию
missing_values = df_sessions.isnull().sum()

# Оставляем только колонки с пропусками и сортируем
missing_sorted = missing_values[missing_values > 0].sort_values(ascending=False)

print("Колонки с пропусками, отсортированные по убыванию:")
print(missing_sorted)


Колонки с пропусками, отсортированные по убыванию:
device_model     1843704
utm_keyword      1082061
device_os        1070138
device_brand      367178
utm_adcontent     335615
utm_campaign      219603
utm_source            97
dtype: int64


Обработка пропуска в поле device_model (замена на "noname")

In [ ]:
df_sessions['device_model'] = df_sessions['device_model'].fillna('noname')

Проверим заполнение данных одновременно по полям device_brand и	device_os

In [ ]:
df_sessions.loc[df_sessions['device_category'] == 'desktop', ['device_brand', 'device_os']].value_counts(dropna=False)

device_brand  device_os
NaN           NaN          248136
              Windows       88291
              Macintosh     24804
              Linux          5063
              (not set)       344
              Chrome OS        83
Apple         NaN              39
              iOS              33
Huawei        NaN              16
(not set)     NaN               8
Xiaomi        Android           6
Samsung       NaN               5
OPPO          NaN               4
Realme        NaN               4
Xiaomi        NaN               4
Google        Android           4
Flexymove     Android           3
Huawei        Android           3
Samsung       Android           3
OnePlus       NaN               2
(not set)     Tizen             2
Beelink       NaN               2
Asus          NaN               1
Realme        Android           1
Nokia         NaN               1
Philips       NaN               1
Name: count, dtype: int64

Обработка пропусков в device_os для устройств Apple

In [ ]:
# Обработка пропусков в device_os для устройств Apple
mask = df_sessions['device_os'].isnull() & (df_sessions['device_brand'] == 'Apple')
df_sessions.loc[mask, 'device_os'] = 'iOS'

# Проверка результата
print("Заполнено записей:", mask.sum())
print(df_sessions.loc[mask, ['device_brand', 'device_os']].head())


Заполнено записей: 343990
   device_brand device_os
5         Apple       iOS
32        Apple       iOS
33        Apple       iOS
34        Apple       iOS
45        Apple       iOS


Переобозначение пропусков в поле device_brand и device_os

In [ ]:
# Маска для строк, где оба поля device_brand и device_os пусты
mask_both_nan = df_sessions['device_brand'].isnull() & df_sessions['device_os'].isnull()

# Заполняем пропуски
df_sessions.loc[mask_both_nan, 'device_brand'] = 'noname_device_brand'
df_sessions.loc[mask_both_nan, 'device_os']    = 'noname_device_os'

# Проверка результата
print(f"Заполнено строк: {mask_both_nan.sum()}")
print(df_sessions.loc[mask_both_nan, ['device_brand', 'device_os']].head())


Заполнено строк: 248500
           device_brand         device_os
63  noname_device_brand  noname_device_os
64  noname_device_brand  noname_device_os
73  noname_device_brand  noname_device_os
74  noname_device_brand  noname_device_os
78  noname_device_brand  noname_device_os


Заполнение device_brand для десктопных ОС

In [ ]:
# Заполнение device_brand = 'PC' для десктопных ОС
desktop_oses = ['Windows', 'Macintosh', 'Linux', 'Chrome OS']
mask_desktop = df_sessions['device_os'].isin(desktop_oses)

df_sessions.loc[mask_desktop, 'device_brand'] = 'PC'

# Проверка результата
print(f"Строк обновлено: {mask_desktop.sum()}")
print(df_sessions.loc[mask_desktop, ['device_os', 'device_brand']].drop_duplicates().head())


Строк обновлено: 118334
       device_os device_brand
28       Windows           PC
206        Linux           PC
225    Macintosh           PC
23600  Chrome OS           PC


In [ ]:
# Маска: device_os пустой, device_brand заполнен и не равен '(not set)'
mask_android = (
    df_sessions['device_os'].isnull() &
    df_sessions['device_brand'].notnull() &
    (df_sessions['device_brand'] != '(not set)')
)

# Заполняем пропуски в device_os значением 'Android'
df_sessions.loc[mask_android, 'device_os'] = 'Android'

# Проверка
print(f"Заполнено строк: {mask_android.sum()}")
print(df_sessions.loc[mask_android, ['device_brand', 'device_os']].drop_duplicates().head())


Заполнено строк: 469881
   device_brand device_os
3        Xiaomi   Android
7       Samsung   Android
47       Huawei   Android
53         Vivo   Android
86           BQ   Android


Удаляем строки с device_brand равным NaN (их количество некритично для выборки)

In [ ]:
# Сохраним исходное количество строк
initial_count = len(df_sessions)

# Фильтруем: оставляем только строки, где device_brand не NaN
df_sessions = df_sessions[df_sessions['device_brand'].notnull()]

# Считаем оставшиеся и удалённые строки
remaining_count = len(df_sessions)
deleted_count = initial_count - remaining_count

print(f"Удалено строк: {deleted_count}")
print(f"Осталось строк: {remaining_count}")


Удалено строк: 344
Осталось строк: 1859698


Удаляем строки с device_brand равным not set  (их количество некритично для выборки)

In [ ]:
# Удаляем все строки, где device_brand == '(not set)'
mask_notset = df_sessions['device_brand'] == '(not set)'
df_sessions = df_sessions.loc[~mask_notset].reset_index(drop=True)

# Проверка: сколько строк было удалено и сколько осталось
print(f"Удалено строк: {mask_notset.sum()}")
print(f"Осталось строк: {len(df_sessions)}")


Удалено строк: 17545
Осталось строк: 1842153


In [ ]:
df_sessions.loc[df_sessions['device_category'] == 'desktop', ['device_brand', 'device_os']].value_counts(dropna=False)

device_brand         device_os       
noname_device_brand  noname_device_os    248136
PC                   Windows              88291
                     Macintosh            24804
                     Linux                 5063
                     Chrome OS               83
Apple                iOS                     72
Huawei               Android                 19
Xiaomi               Android                 10
Samsung              Android                  8
Realme               Android                  5
Google               Android                  4
OPPO                 Android                  4
Flexymove            Android                  3
Beelink              Android                  2
OnePlus              Android                  2
Asus                 Android                  1
Nokia                Android                  1
Philips              Android                  1
Name: count, dtype: int64

Обработка пропуска в поле utm_keyword

In [ ]:
# 1. Маска строк с непустым utm_campaign
mask_campaign = df_sessions['utm_campaign'].notnull()

# 2. Всего таких строк
total_campaign = mask_campaign.sum()

# 3. Среди них — сколько с непустым utm_keyword
with_keyword = df_sessions.loc[mask_campaign, 'utm_keyword'].notnull().sum()

# 4. Сколько без utm_keyword
without_keyword = total_campaign - with_keyword

print(f'Всего строк с utm_campaign: {total_campaign}')
print(f'Из них строк с utm_keyword: {with_keyword}')
print(f'Строк без utm_keyword при существующем utm_campaign: {without_keyword}')

# 5. Процент неполных записей
pct = without_keyword / total_campaign * 100
print(f'Процент строк, где utm_campaign есть, а utm_keyword нет: {pct:.2f}%')


Всего строк с utm_campaign: 1624036
Из них строк с utm_keyword: 692895
Строк без utm_keyword при существующем utm_campaign: 931141
Процент строк, где utm_campaign есть, а utm_keyword нет: 57.33%


Таким образом, utm_keyword присутствует не всегда, поэтому его можно заменить на значение "without_utm_keyword"


In [ ]:
df_sessions['utm_keyword'] = df_sessions['utm_keyword'].fillna('without_utm_keyword')

Попробуем обработать пропуски в utm_source.
Предположим, что пропуски по полю utm_source можно подставить, если клиент (client_id) с такой же сессией (session_id) имеет заполненной колонку utm_source

In [ ]:
# 1) Для каждой группы (client_id, session_id) найдём первое ненулевое значение utm_source
## first_sources = (
##    df_sessions
##    .groupby(['client_id', 'session_id'])['utm_source']
##    .transform(lambda col: col.dropna().iloc[0] if col.dropna().any() else np.nan)
##)

# 2) Заполним пропуски в utm_source этим значением
##df_sessions['utm_source'] = df_sessions['utm_source'].fillna(first_sources)

# Проверка:
##print(df_sessions['utm_source'].isnull().sum(), "— осталось пропусков")

Пропусков осталось то же количество (97), т.к. не оказалом одинаковых сочетаний client_id и session_id.

Тогда удалим строки с пропусками в utm_source  (их количество некритично для выборки)


In [ ]:
# Сохраняем исходное количество строк
initial_count = len(df_sessions)

# Удаляем строки с пропущенным utm_source
df_sessions = df_sessions[df_sessions['utm_source'].notnull()]

# Считаем удалённые и оставшиеся строки
remaining_count = len(df_sessions)
deleted_count = initial_count - remaining_count

print(f"Удалено строк с пропущенным utm_source: {deleted_count}")
print(f"Осталось строк: {remaining_count}")


Удалено строк с пропущенным utm_source: 97
Осталось строк: 1842056


Проверим оставшиеся пропуски

In [ ]:
# Проверка количества пропусков в каждой колонке
missing_values = df_sessions.isnull().sum()
print("Количество пропусков (NaN/None) в каждой колонке:")
print(missing_values[missing_values > 0])  # Показываем только колонки с пропусками

Количество пропусков (NaN/None) в каждой колонке:
utm_campaign     218117
utm_adcontent    333613
dtype: int64


Проставим значение without_utm_campaign в поле utm_campaign, в которых есть пропуски

In [ ]:
# Маска строк с пропущенным utm_campaign
mask_no_campaign = df_sessions['utm_campaign'].isnull()

# Заполняем их значением 'without_utm_campaign'
df_sessions.loc[mask_no_campaign, 'utm_campaign'] = 'without_utm_campaign'

# Проверка результата
print(f"Заполнено строк: {mask_no_campaign.sum()}")
print(df_sessions['utm_campaign'].value_counts().head())


Заполнено строк: 218117
utm_campaign
LTuZkdKfxRGVceoWkVyg    460503
LEoPHuyFvzoNfnzGgfcd    319490
FTjNLDyTrXaWYgZymFkV    243917
without_utm_campaign    218117
gecBYcKZCPMcVYdSSzKP    132505
Name: count, dtype: int64


Проставим значение without_utm_adcontent в поле utm_adcontent, в которых есть пропуски

In [ ]:
# Маска строк с пропущенным utm_campaign
mask_no_adconten = df_sessions['utm_adcontent'].isnull()

# Заполняем их значением 'without_utm_campaign'
df_sessions.loc[mask_no_adconten, 'utm_adcontent'] = 'without_utm_adcontent'

# Проверка результата
print(f"Заполнено строк: {mask_no_adconten.sum()}")
print(df_sessions['utm_adcontent'].value_counts().head())

Заполнено строк: 333613
utm_adcontent
JNHcPlZPxEMWDnRiyoBf     998250
without_utm_adcontent    333613
vCIpmpaGBnIQhyYNkXqp     177757
xhoenQgDQsgfEPYNPwKO     111268
PkybGvWbaqORmxjNunqZ      59302
Name: count, dtype: int64


Добавим новую колонку has_utm_campaign с бинарными признаками, если рекламная кампания 1 - была, 2 - не было

In [ ]:
# Добавляем отдельный бинарный признак: 1, если utm_campaign не пустой, 0 если пустой
df_sessions['has_utm_campaign'] = df_sessions['utm_campaign'].notna().astype(int)

# Проверка результата
print(df_sessions[['utm_campaign', 'has_utm_campaign']].head())
print("Уникальные значения в has_utm_campaign:", df_sessions['has_utm_campaign'].unique())



           utm_campaign  has_utm_campaign
0  LEoPHuyFvzoNfnzGgfcd                 1
1  FTjNLDyTrXaWYgZymFkV                 1
2  LEoPHuyFvzoNfnzGgfcd                 1
3  without_utm_campaign                 1
4  without_utm_campaign                 1
Уникальные значения в has_utm_campaign: [1]


Контрольная проверка пропусков в датасете

In [ ]:
# Проверка количества пропусков в каждой колонке
missing_values = df_sessions.isnull().sum()
print("Количество пропусков (NaN/None) в каждой колонке:")
print(missing_values[missing_values > 0])  # Показываем только колонки с пропусками

Количество пропусков (NaN/None) в каждой колонке:
Series([], dtype: int64)


Обработка типов данных в датасете df_sessions

In [ ]:
# Просмотр колонок датасета
print(df_sessions.columns)

Index(['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'utm_keyword', 'device_category', 'device_os', 'device_brand',
       'device_model', 'device_screen_resolution', 'device_browser',
       'geo_country', 'geo_city', 'has_utm_campaign'],
      dtype='object')


In [ ]:
df_sessions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1842056 entries, 0 to 1842152
Data columns (total 19 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   session_id                object
 1   client_id                 object
 2   visit_date                object
 3   visit_time                object
 4   visit_number              int64 
 5   utm_source                object
 6   utm_medium                object
 7   utm_campaign              object
 8   utm_adcontent             object
 9   utm_keyword               object
 10  device_category           object
 11  device_os                 object
 12  device_brand              object
 13  device_model              object
 14  device_screen_resolution  object
 15  device_browser            object
 16  geo_country               object
 17  geo_city                  object
 18  has_utm_campaign          int64 
dtypes: int64(2), object(17)
memory usage: 281.1+ MB


In [ ]:
# Изменение настроек типов для отдельных колонок
dtype_settings = {
    'client_id': 'str',
    'session_id': 'str',
}

In [ ]:
# Проверка изменения настроек типов
print(df_sessions['client_id'].dtype)  # должно быть object или str
print(df_sessions['client_id'].head())  # провека первых значений

object
0    2108382700.1637753791
1     210838531.1636867288
2    2108385331.1640648523
3    2108385564.1622255328
4    2108385564.1622255328
Name: client_id, dtype: object


In [ ]:
# Преобразуем visit_date в datetime (дата)
df_sessions['visit_date'] = pd.to_datetime(
    df_sessions['visit_date'],
    format='%Y-%m-%d',  # или оставьте без format, если формат может варьироваться
    errors='raise'
)

# Преобразуем строку вида "HH:MM:SS" в timedelta (время с полуночи)
df_sessions['visit_time'] = pd.to_timedelta(df_sessions['visit_time'], errors='coerce')

# Проверим типы
print(df_sessions[['visit_date','visit_time']].dtypes)

visit_date     datetime64[ns]
visit_time    timedelta64[ns]
dtype: object


In [ ]:
df_sessions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1842056 entries, 0 to 1842152
Data columns (total 19 columns):
 #   Column                    Dtype          
---  ------                    -----          
 0   session_id                object         
 1   client_id                 object         
 2   visit_date                datetime64[ns] 
 3   visit_time                timedelta64[ns]
 4   visit_number              int64          
 5   utm_source                object         
 6   utm_medium                object         
 7   utm_campaign              object         
 8   utm_adcontent             object         
 9   utm_keyword               object         
 10  device_category           object         
 11  device_os                 object         
 12  device_brand              object         
 13  device_model              object         
 14  device_screen_resolution  object         
 15  device_browser            object         
 16  geo_country               object         

# **Обработка датасета df_hits**

In [ ]:
# Просмотр колонок датасета
print(df_hits.columns)

Index(['session_id', 'hit_date', 'hit_time', 'hit_number', 'hit_type',
       'hit_referer', 'hit_page_path', 'event_category', 'event_action',
       'event_label', 'event_value'],
      dtype='object')


In [ ]:
# Подсчёт пропусков и сортировка по убыванию
missing_values = df_hits.isnull().sum()

# Оставляем только колонки с пропусками и сортируем
missing_sorted = missing_values[missing_values > 0].sort_values(ascending=False)

print("Колонки с пропусками, отсортированные по убыванию:")
print(missing_sorted)


Колонки с пропусками, отсортированные по убыванию:
event_value    15726470
hit_time        9160322
hit_referer     6274804
event_label     3760184
dtype: int64


Обработаем пропуски

In [ ]:
# 0. Смотрим изначальное число строк
initial_hits = len(df_hits)

# 1. Заполняем event_value нулями
df_hits['event_value'] = df_hits['event_value'].fillna(0)

# 2. Преобразуем hit_time в timedelta и заполняем 0
df_hits['hit_time'] = pd.to_timedelta(df_hits['hit_time'], errors='coerce')
df_hits['hit_time'] = df_hits['hit_time'].fillna(pd.Timedelta(seconds=0))

# 3. Заполняем текстовые поля
df_hits['hit_referer']  = df_hits['hit_referer'].fillna('unknown')
df_hits['event_label']  = df_hits['event_label'].fillna('unknown')

# 4. Проверяем результат
missing_after = df_hits.isnull().sum()
print("Пропуски после заполнения:\n", missing_after[missing_after > 0])

# 5. Сохраняем очищенный df_hits
print(f"Строк обработано: {initial_hits} → {len(df_hits)} (не удалили ни одной)")

Пропуски после заполнения:
 Series([], dtype: int64)
Строк обработано: 15726470 → 15726470 (не удалили ни одной)


In [ ]:
# Подсчёт пропусков и сортировка по убыванию
missing_values = df_hits.isnull().sum()

# Оставляем только колонки с пропусками и сортируем
missing_sorted = missing_values[missing_values > 0].sort_values(ascending=False)

print("Колонки с пропусками, отсортированные по убыванию:")
print(missing_sorted)

Колонки с пропусками, отсортированные по убыванию:
Series([], dtype: int64)


Ограничим датасет до 2х колонок: session_id и event_action

In [ ]:
# Оставляем только нужные колонки
df_hits = df_hits[['session_id', 'event_action']]

In [ ]:
print(df_hits.columns)  # Должно вывести: Index(['session_id', 'event_action'], dtype='object')

Index(['session_id', 'event_action'], dtype='object')


### **Объедение 2х датасетов**

In [ ]:
# Проверка уникальности session_id в обоих датасетах
print(f"Уникальных session_id в датасете hits: {df_hits['session_id'].nunique()}")
print(f"Уникальных session_id в датасете sessions: {df_sessions['session_id'].nunique()}")

# Проверка наличия пропусков
print(f"Пропуски в session_id (датасет hits): {df_hits['session_id'].isnull().sum()}")
print(f"Пропуски в session_id (датасет sessions): {df_sessions['session_id'].isnull().sum()}")

Уникальных session_id в датасете hits: 1734610
Уникальных session_id в датасете sessions: 1842056
Пропуски в session_id (датасет hits): 0
Пропуски в session_id (датасет sessions): 0


In [ ]:
# Объединение датасетов hits и sessions по session_id (left join, чтобы сохранить все события)

df = pd.merge(
    left=df_hits,
    right=df_sessions,
    on='session_id',
    how='left',
    suffixes=('_hit', '_session')
)

Обработаем дубликаты в объединенном датасете

In [ ]:
print(f"До удаления: {df.shape[0]} строк")
df = df.drop_duplicates(subset=['session_id'])

До удаления: 15726470 строк


In [ ]:
# Удаление дубликатов
df = df.drop_duplicates(subset=['session_id'])

In [ ]:
print(f"После удаления: {df.shape[0]} строк")
print(f"Удалено дубликатов: {df.duplicated(subset=['session_id']).sum()}")

После удаления: 1734610 строк
Удалено дубликатов: 0


In [ ]:
df.drop_duplicates(subset=['session_id']).to_csv(
    'cleaned_sessions_new.csv',  # Имя файла
    index=False              # Не сохранять индексы
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Проектный практикум (хакатон)/cleaned_sessions_new1.csv'
df.drop_duplicates(subset=['session_id']).to_csv(path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1734610 entries, 0 to 15726432
Data columns (total 20 columns):
 #   Column                    Dtype          
---  ------                    -----          
 0   session_id                object         
 1   event_action              object         
 2   client_id                 object         
 3   visit_date                datetime64[ns] 
 4   visit_time                timedelta64[ns]
 5   visit_number              float64        
 6   utm_source                object         
 7   utm_medium                object         
 8   utm_campaign              object         
 9   utm_adcontent             object         
 10  utm_keyword               object         
 11  device_category           object         
 12  device_os                 object         
 13  device_brand              object         
 14  device_model              object         
 15  device_screen_resolution  object         
 16  device_browser            object        